In [ ]:
# pip install playwright bs4
# python -m playwright install
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "playwright", "install", "chromium"])

## 2021: 20201101 - 20220313

In [1]:
# pip install playwright bs4 pandas python-dateutil
# python -m playwright install   # only once

import asyncio
import os
from datetime import datetime, timedelta
from typing import List
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PWTimeout

BASE_URL = (
    "https://barttorvik.com/trank.php"
    "?year={year}&sort=&hteam=&t2value=&conlimit=All&state=All"
    "&begin={begin}&end={end}&top=0&revquad=0&quad=5&venue=All&type=All&mingames=0#"
)

# ---------- HTML fetch ----------

async def goto_and_get_html(page, url: str, table_selector: str = "table", timeout_ms: int = 30000) -> str:
    await page.goto(url, wait_until="domcontentloaded", timeout=timeout_ms)
    try:
        await page.wait_for_selector(table_selector, timeout=20000)
    except PWTimeout:
        await page.wait_for_load_state("networkidle", timeout=10000)

    # if we’re still on the verification page, wait a bit
    for _ in range(6):
        html = await page.content()
        if "Verifying your browser" not in html and "js_test_submitted" not in html:
            break
        await asyncio.sleep(1)

    return await page.content()

# ---------- Table parsing ----------

def parse_first_table(html: str) -> List[List[str]]:
    soup = BeautifulSoup(html, "html.parser")
    table = soup.select_one("table")
    if not table:
        return []
    rows = []
    for tr in table.select("tr"):
        cells = [c.get_text(strip=True) for c in tr.select("th, td")]
        if cells:
            rows.append(cells)
    return rows


def rows_to_dataframe(rows: List[List[str]]) -> pd.DataFrame:
    """Convert raw scraped rows into a DataFrame"""
    if len(rows) < 3:
        return pd.DataFrame()
    columns = rows[1]  # second row = headers
    data = rows[2:]
    max_len = len(columns)
    norm = [r[:max_len] + ([""] * (max_len - len(r))) for r in data]
    df = pd.DataFrame(norm, columns=columns)
    return df

# ---------- Orchestrator ----------

async def scrape_barttorvik_daily(
    year: int = 2021,
    begin: str = "20201101",
    end: str = "20210313",
    output_dir: str = "daily_csvs",
    master_csv: str = "barttorvik_2021_all.csv",
    table_selector: str = "table",
    headless: bool = True,
    pause_sec: float = 3.8
):
    os.makedirs(output_dir, exist_ok=True)
    start_dt = datetime.strptime(begin, "%Y%m%d")
    final_dt = datetime.strptime(end, "%Y%m%d")

    first_write = not os.path.exists(master_csv)

    async with async_playwright() as p:
        browser = await p.chromium.launch(channel="chrome", headless=headless)
        context = await browser.new_context(
            user_agent=("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                        "AppleWebKit/537.36 (KHTML, like Gecko) "
                        "Chrome/119.0.0.0 Safari/537.36"),
            locale="en-US",
            timezone_id="America/New_York",
        )
        page = await context.new_page()

        dt = start_dt
        total_rows = 0
        while dt <= final_dt:
            end_str = dt.strftime("%Y%m%d")
            url = BASE_URL.format(year=year, begin=begin, end=end_str)

            try:
                html = await goto_and_get_html(page, url, table_selector=table_selector)
                rows = parse_first_table(html)
                df = rows_to_dataframe(rows)

                if not df.empty:
                    df.insert(0, "Date", end_str)

                    # write individual daily file
                    daily_path = os.path.join(output_dir, f"barttorvik_{end_str}.csv")
                    df.to_csv(daily_path, index=False)
                    print(f"✔️  {end_str}: saved {len(df)} rows to {daily_path}")

                    # append to master CSV
                    if first_write:
                        df.to_csv(master_csv, index=False)
                        first_write = False
                    else:
                        df.to_csv(master_csv, mode="a", header=False, index=False)

                    total_rows += len(df)
                else:
                    print(f"⚠️  {end_str}: no data (empty table)")

            except Exception as e:
                print(f"❌ {end_str}: ERROR {e}")

            await asyncio.sleep(pause_sec)
            dt += timedelta(days=1)

        await browser.close()

    print(f"\n✅ Done! {total_rows} total rows saved across days.")

# ---------- Run ----------
# In Jupyter or async environment:
await scrape_barttorvik_daily(
    year=2025,
    begin="20241103",
    end="20250318",
    output_dir="daily_csvs_2025",
    master_csv="barttorvik_2025_all.csv"
)


✔️  20241103: saved 1 rows to daily_csvs_2025/barttorvik_20241103.csv
✔️  20241104: saved 242 rows to daily_csvs_2025/barttorvik_20241104.csv
✔️  20241105: saved 258 rows to daily_csvs_2025/barttorvik_20241105.csv
✔️  20241106: saved 288 rows to daily_csvs_2025/barttorvik_20241106.csv
✔️  20241107: saved 301 rows to daily_csvs_2025/barttorvik_20241107.csv
✔️  20241108: saved 341 rows to daily_csvs_2025/barttorvik_20241108.csv
✔️  20241109: saved 367 rows to daily_csvs_2025/barttorvik_20241109.csv
✔️  20241110: saved 372 rows to daily_csvs_2025/barttorvik_20241110.csv
✔️  20241111: saved 375 rows to daily_csvs_2025/barttorvik_20241111.csv
✔️  20241112: saved 377 rows to daily_csvs_2025/barttorvik_20241112.csv
✔️  20241113: saved 377 rows to daily_csvs_2025/barttorvik_20241113.csv
✔️  20241114: saved 377 rows to daily_csvs_2025/barttorvik_20241114.csv
✔️  20241115: saved 377 rows to daily_csvs_2025/barttorvik_20241115.csv
✔️  20241116: saved 378 rows to daily_csvs_2025/barttorvik_2024111